In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
matplotlib.style.use('seaborn-notebook')
plt.rcParams['figure.figsize'] = (14.0, 11.0)
%load_ext autoreload
%autoreload 2

ImportError: No module named 'matplotlib'

In [1]:
import process as p
import dmc

In [2]:
df = p.processed_data()

OSError: File b'data/datacup-out-manual.csv' does not exist

In [1]:
start, end, split = pd.Timestamp('2014-1-1'), pd.Timestamp('2014-12-31'), pd.Timestamp('2014-10-1')
df.orderDate = pd.to_datetime(df.orderDate)
mask = (df.orderDate >= start) & (df.orderDate <= end)
df_full = df[mask]
df_train = df_full[df_full.orderDate < split]
df_class = df_full[df_full.orderDate >= split]
len(df_train), len(df_class), len(df_full), len(df)

NameError: name 'pd' is not defined

In [2]:
X, Y, fts = dmc.transformation.transform_preserving_header(
    df_full, scaler=dmc.normalization.scale_features, binary_target=True)

NameError: name 'dmc' is not defined

In [6]:
X_train, Y_train = X[:5000], Y[:5000]
X_class, Y_class = X[5000:20000], Y[5000:20000]

In [15]:
res_tree = dmc.evaluation.evaluate_features_leaving_one_out(
    X_train, Y_train, X_class, Y_class, fts, dmc.classifiers.DecisionTree)
res_bayes = dmc.evaluation.evaluate_features_leaving_one_out(
    X_train, Y_train, X_class, Y_class, fts, dmc.classifiers.NaiveBayes)
res_svm = dmc.evaluation.evaluate_features_leaving_one_out(
    X_train, Y_train, X_class, Y_class, fts, dmc.classifiers.SVM)